In [7]:
import requests
import pandas as pd
import time
import os
from urllib.request import urlretrieve

# --- CONFIG ---
API_KEY = 'd8ae884deab9fe57f68e1f96fbd8aad3'  # Replace with your actual API key
BASE_URL = 'https://api.themoviedb.org/3'
POSTER_BASE = 'https://image.tmdb.org/t/p/w500'
PAGES = 5  # Number of pages to fetch
SAVE_DIR = 'tmdb_data'  # Directory to save all files

# Create save directory if it doesn't exist
os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(os.path.join(SAVE_DIR, 'posters'), exist_ok=True)

def get_genre_mapping():
    """Fetch TV genre mapping from TMDB"""
    url = f"{BASE_URL}/genre/tv/list"
    params = {'api_key': API_KEY}
    res = requests.get(url, params=params)
    res.raise_for_status()
    return {g['id']: g['name'] for g in res.json()['genres']}

def download_image(url, save_path):
    """Download and save an image from URL"""
    try:
        urlretrieve(url, save_path)
        return True
    except Exception as e:
        print(f"Failed to download {url}: {str(e)}")
        return False

def get_tv_shows(pages=1):
    """Fetch TV shows from TMDB"""
    shows = []
    url = f"{BASE_URL}/discover/tv"
    
    for page in range(1, pages + 1):
        print(f"Fetching page {page}/{pages}")
        params = {
            'api_key': API_KEY,
            'language': 'en-US',
            'sort_by': 'popularity.desc',
            'page': page,
        }
        
        try:
            res = requests.get(url, params=params)
            res.raise_for_status()
            data = res.json()
            
            for show in data['results']:
                poster_path = show.get('poster_path')
                poster_url = f"{POSTER_BASE}{poster_path}" if poster_path else None
                poster_file = os.path.join('posters', f"{show['id']}.jpg") if poster_path else None
                
                if poster_url:
                    download_image(poster_url, os.path.join(SAVE_DIR, poster_file))
                
                shows.append({
                    'id': show['id'],
                    'title': show['name'],
                    'overview': show.get('overview'),
                    'popularity': show.get('popularity'),
                    'vote_average': show.get('vote_average'),
                    'first_air_date': show.get('first_air_date'),
                    'poster_file': poster_file,  # Local path to the downloaded image
                    'genres': show.get('genre_ids', []),
                })
                
            time.sleep(0.3)  # Respect rate limits
        except Exception as e:
            print(f"Error on page {page}: {str(e)}")
            continue
            
    return shows

def main():
    try:
        print("Starting TMDB TV show data collection...")
        
        # Get genre mapping
        genre_map = get_genre_mapping()
        
        # Get TV shows
        shows = get_tv_shows(PAGES)
        
        # Create DataFrame
        df = pd.DataFrame(shows)
        
        # Map genre IDs to names
        df['genres'] = df['genres'].apply(
            lambda ids: ', '.join([genre_map.get(i, 'Unknown') for i in ids])
        )
        
        # Save metadata to CSV
        csv_path = os.path.join(SAVE_DIR, 'tv_shows_metadata.csv')
        df.to_csv(csv_path, index=False)
        print(f"\nSuccessfully saved {len(df)} shows to:")
        print(f"- Metadata: {csv_path}")
        print(f"- Posters: {os.path.join(SAVE_DIR, 'posters')}")
        
        # Display sample
        print("\nSample data:")
        print(df[['id', 'title', 'genres', 'vote_average']].head(3).to_string(index=False))
        
    except Exception as e:
        print(f"\nScript failed: {str(e)}")

if __name__ == "__main__":
    main()

Starting TMDB TV show data collection...
Fetching page 1/5
Fetching page 2/5
Fetching page 3/5
Fetching page 4/5
Fetching page 5/5

Successfully saved 100 shows to:
- Metadata: tmdb_data/tv_shows_metadata.csv
- Posters: tmdb_data/posters

Sample data:
   id                                   title       genres  vote_average
 2261 The Tonight Show Starring Johnny Carson         Talk         7.410
22980 Watch What Happens Live with Andy Cohen Talk, Comedy         5.000
59941  The Tonight Show Starring Jimmy Fallon Comedy, Talk         5.865
